In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from notebook_utils import NotebookUtilities
from scrape_utils import WebScrapingUtilities
from pandas import DataFrame
import os
import os.path as osp
import random
import re
import logging

nu = NotebookUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)
wsu = WebScrapingUtilities(
    s=nu,
    secrets_json_path=osp.abspath(osp.join(nu.data_folder, 'secrets', 'itm_secrets.json'))
)


# Parse Domain Documents for Entities

Downloaded all documents from https://nextcentury.atlassian.net/wiki/spaces/ITMC/pages/2991849482/Domain+Documents and converted them all to PDF files and stored them in the data folder.

In [3]:

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

pdf_folder = '../data/Domain_Knowledge'
black_list = ['.ipynb_checkpoints', '$Recycle.Bin', '.git']


## Option 1: Use a Hugging Face NER model

In [4]:

from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Named entity recognition pipeline, passing in a specific model and tokenizer
model = AutoModelForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
token_classifier = pipeline('ner', model=model, tokenizer=tokenizer)

# Example usage
sentence = 'Barack Obama was the 44th President of the United States.'
tokens = token_classifier(sentence)
print(tokens)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER', 'score': 0.9988381, 'index': 1, 'word': 'Barack', 'start': 0, 'end': 6}, {'entity': 'I-PER', 'score': 0.9994398, 'index': 2, 'word': 'Obama', 'start': 7, 'end': 12}, {'entity': 'I-LOC', 'score': 0.9983613, 'index': 10, 'word': 'United', 'start': 43, 'end': 49}, {'entity': 'I-LOC', 'score': 0.9920671, 'index': 11, 'word': 'States', 'start': 50, 'end': 56}]



## Option 2: Use SpaCy

In [5]:

import spacy

# Load the spaCy model
try: nlp = spacy.load('en_core_web_sm')
except OSError as e:
    print(str(e).strip())
    command_str = f'{sys.executable} -m spacy download en_core_web_sm --quiet'
    print(command_str)
    !{command_str}
    nlp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()

# Example usage
sentence = 'Barack Obama was the 44th President of the United States.'
doc = nlp(sentence)
print([{'text': word.text, 'tag_': word.tag_, 'ent_type_': word.ent_type_, 'pos_': word.pos_} for word in doc])
print([{'text': ent.text, 'label_': ent.label_} for ent in doc.ents])

[{'text': 'Barack', 'tag_': 'NNP', 'ent_type_': 'PERSON', 'pos_': 'PROPN'}, {'text': 'Obama', 'tag_': 'NNP', 'ent_type_': 'PERSON', 'pos_': 'PROPN'}, {'text': 'was', 'tag_': 'VBD', 'ent_type_': '', 'pos_': 'AUX'}, {'text': 'the', 'tag_': 'DT', 'ent_type_': '', 'pos_': 'DET'}, {'text': '44th', 'tag_': 'JJ', 'ent_type_': 'ORDINAL', 'pos_': 'ADJ'}, {'text': 'President', 'tag_': 'NNP', 'ent_type_': '', 'pos_': 'PROPN'}, {'text': 'of', 'tag_': 'IN', 'ent_type_': '', 'pos_': 'ADP'}, {'text': 'the', 'tag_': 'DT', 'ent_type_': 'GPE', 'pos_': 'DET'}, {'text': 'United', 'tag_': 'NNP', 'ent_type_': 'GPE', 'pos_': 'PROPN'}, {'text': 'States', 'tag_': 'NNP', 'ent_type_': 'GPE', 'pos_': 'PROPN'}, {'text': '.', 'tag_': '.', 'ent_type_': '', 'pos_': 'PUNCT'}]
[{'text': 'Barack Obama', 'label_': 'PERSON'}, {'text': '44th', 'label_': 'ORDINAL'}, {'text': 'the United States', 'label_': 'GPE'}]



## Extract the text from PDFs and load it into documents

In [6]:

# Get text from PDFs
if nu.pickle_exists('domain_knowledge_sentences_dict'):
    domain_knowledge_sentences_dict = nu.load_object('domain_knowledge_sentences_dict')
else:
    from PyPDF2 import PdfReader
    def convert(file_path, verbose=False):
        """
        Convert PDF, return its text content as a string
        """
        text = ''
        with open(file_path, 'rb') as file:
            pdf_reader = PdfReader(file)
            for page_number in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_number]
                text += page.extract_text()
        if verbose: print(f'Text length for {file_path} is {len(text):,} characters.')

        return text
    domain_knowledge_sentences_dict = {}
    for sub_directory, directories_list, files_list in os.walk(pdf_folder):
        if all(map(lambda x: x not in sub_directory, black_list)):
            for file_name in files_list:
                if file_name.endswith('.pdf'):
                    file_path = osp.join(sub_directory, file_name)
                    text = convert(file_path)
                    domain_knowledge_sentences_dict[file_path] = text
    nu.store_objects(domain_knowledge_sentences_dict=domain_knowledge_sentences_dict)

In [7]:

import pandas as pd

# load documents
if nu.csv_exists('domain_doc_ners_df'): domain_doc_ners_df = nu.load_data_frames(domain_doc_ners_df='domain_doc_ners_df')['domain_doc_ners_df']
else:
    entities = []
    for file_path, text in domain_knowledge_sentences_dict.items():
        text_length = len(text)
        # print(f'Text length for {file_path} is {text_length:,} characters.')
        
        # Prepare to join subword tokens back together and keep track of entity and score
        output_words = []
        current_word = ''
        current_entities = []
        current_scores = []
        
        # Extract metadata from entity recognition pipeline and add it as a row dictionary to the entities rows list
        tokens = token_classifier(text)
        for metadata_dict in tokens:
            current_entities.append(metadata_dict['entity'])
            current_scores.append(metadata_dict['score'])
            if metadata_dict['word'].startswith('##'):
                current_word += metadata_dict['word'][2:]
            else:
                
                # Take the mode of entities and average of scores for the current_word
                if current_word:
                    mode_entity = pd.Series(current_entities).mode().tolist()[0]
                    mean_score = pd.Series(current_scores).mean()
                    entity_tuple = (current_word, mode_entity, mean_score)
                    output_words.append(entity_tuple)
                    current_word = ""
                    current_entities = []
                    current_scores = []
                
                entity_tuple = (metadata_dict['word'], metadata_dict['entity'], metadata_dict['score'])
                output_words.append(entity_tuple)
        
        # Take the mode of entities for the last current_word
        if current_word:
            mode_entity = pd.Series(current_entities).mode().tolist()[0]
            mean_score = pd.Series(current_scores).mean()
            entity_tuple = (current_word, mode_entity, mean_score)
            output_words.append(entity_tuple)
        
        for word, entity, score in output_words:
            metadata_dict = {'bert_word': word, 'bert_entity': entity, 'bert_score': score, 'file_path': file_path}
            entities.append(metadata_dict)

        # Extract SpaCy named entities and add them as a row dictionary to the entities rows list
        if text_length <= nlp.max_length:
            doc = nlp(text)
            entities.extend([
                {'file_path': file_path, 'nlp_word': word.text, 'nlp_tag': word.tag_, 'nlp_type': word.ent_type_, 'nlp_pofs': word.pos_}
                for word in doc
            ])
            entities.extend([
                {'file_path': file_path, 'ent_phrase': ent.text, 'ent_type': ent.label_, 'ent_start': ent.start_char, 'ent_end': ent.end_char}
                for ent in doc.ents
            ])
    domain_doc_ners_df = DataFrame(entities)
    nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/domain_doc_ners_df.pkl.
No pickle exists - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv.


In [8]:

# Check that all the subword tokens are gone
print(domain_doc_ners_df.columns.tolist())
mask_series = domain_doc_ners_df.bert_word.map(lambda x: str(x).startswith('##'))
df = domain_doc_ners_df[mask_series]
if (df.shape[0] >= 4): display(df.sample(4).dropna(axis='columns', how='all').T)

['bert_word', 'bert_entity', 'bert_score', 'file_path', 'nlp_word', 'nlp_tag', 'nlp_type', 'nlp_pofs', 'ent_phrase', 'ent_type', 'ent_start', 'ent_end']


In [9]:

# Check that you indeed have only strings among the BERT entities
import numpy as np

mask_series = domain_doc_ners_df.bert_entity.isnull()
sorted(domain_doc_ners_df[~mask_series].bert_entity.tolist(), key=lambda x: len(str(x)), reverse=False)[:10]

['I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG']


## Explore the entity columns

In [21]:

for entity, entity_df in domain_doc_ners_df.groupby('bert_entity'):
    texts_list = sorted(entity_df.bert_word.unique())
    print()
    print(entity, random.sample(texts_list, min(len(texts_list), 50)))


I-LOC ['Drive', 'St', 'OLSBY', 'Gordon', 'Road', 'Asia', 'Kansas', 'New', 'anile', '21st', 'M', 'Philippines', 'Sai', 'Einstein', 'Yard', 'Falls', 'Pacific', 'Baltimore', 'Vietnam', 'Lee', 'Seoul', 'Val', 'Pearl', 'First', 'esda', 'San', 'ébert', 'sburg', 'ANET', 'Spain', 'Delaware', 'Glen', 'eutral', 'Zealand', 'ville', 'ERICA', 'Det', 'inia', 'Street', 'State', 'rgent', 'Philadelphia', 'and', 'Antonio', 'BC', 'MA', 'g', 'Washington', 'Charleston', 'J']

I-MISC ['iformed', 'Nazi', '-', 'Military', 'States', 'Vietnamese', 'AZI', 'AC', 'Model', 'UMANITARIAN', 'Swan', 'Democracy', 'War', 'UM', 'A', 'LITYSICI', 'Territories', 'LITID', 'Con', 'Bell', 'o', 'Jew', 'Prussian', 'OHC', 'E', 'G', 'Napoleonic', 'Western', 'Gulf', 'Experiment', 'sville', 'ins', 'Pat', 'Desert', 'L', 'UMANITARIA', 'J', 'Semi', 'mmde', 'Walt', 'All', 'mitatus', 'drome', 'Andrew', 'chau', 'Social', 'With', 'AT', 'SE', 'Art']

I-ORG ['NKLIN', 'OLLMAR', 'P', 'Ping', 'Division', 'LITEHABIL', 'II', 'Command', 'Gold', 'A


## Explore the entity type columns

In [11]:

words_list = sorted([str(w) for w in domain_doc_ners_df.nlp_type.unique()])
print(random.sample(words_list, min(len(words_list), 100)))

['MONEY', 'PERSON', 'LAW', 'PERCENT', 'PRODUCT', 'WORK_OF_ART', 'ORDINAL', 'DATE', 'LOC', 'EVENT', 'ORG', 'GPE', 'FAC', 'TIME', 'NORP', 'QUANTITY', 'CARDINAL', 'LANGUAGE', 'nan']


In [20]:

for entity, entity_df in domain_doc_ners_df.groupby('nlp_type'):
    texts_list = sorted([str(w) for w in entity_df.nlp_word.unique()])
    print()
    print(entity, random.sample(texts_list, min(len(texts_list), 50)))


CARDINAL ['267', '5%.33', '144–145', '812', '103–124', '1918;47:618', 'approximately', '185', '783', '18.23.4', '775', '1977;4:41–50', '355', '2009;24(1):51–56', '1125/', '2000;May(374):36–46', '52–88', '1.11.4', '1987;17(2):119–132', '1985;98(2):310', '2007;87:157–184', '2007;172(11', '14c(2)(a)(i', '264', '2010;76:71–81', '1984;149:537–541', 'scriptural-', '581', 'three', '2002;24(2):228–247', '2005;9:416–422', '2.2', '45–73', '749', '25.9', '289', '3.5', '1–23', '2013;13(1):77–85', '441', '2.5.4', '209–243', '297', '1984;44(5):24–26', '1993;38(3):542–553', '219.116', '839', 'additional', '1998;24(2):32', '2003;74(4):309–314']

DATE ['1127', 'prior', '2012–2017', '1995;274(10):826–829', 'eighteenth', '2074', '74%,22', '640', '48', '1939', '1989;11(4):7–9', '3331', 'one', '410', '2020', '2381', '1825', '21204', '69', '1996:1', '7', '39', 'school', '1864', 'coming', '1978;8(2', '.', '1991', '1986;8(2):63–73', '1892', 'Middle', 'sixties', '4301', '13', 'Only', '77', '1101', '1938–1978'

In [13]:

domain_doc_ners_df.ent_type.unique()

array([nan, 'CARDINAL', 'DATE', 'ORG', 'GPE', 'PERSON', 'EVENT',
       'ORDINAL', 'FAC', 'PERCENT', 'TIME', 'WORK_OF_ART', 'LOC',
       'PRODUCT', 'NORP', 'LAW', 'MONEY', 'QUANTITY', 'LANGUAGE'],
      dtype=object)

In [19]:

for entity_type, entity_type_df in domain_doc_ners_df.groupby('ent_type'):
    print()
    mask_series = entity_type_df.ent_phrase.isnull()
    texts_list = sorted(entity_type_df[~mask_series].ent_phrase.unique())
    print(entity_type, random.sample(texts_list, min(len(texts_list), 50)))


CARDINAL ['300', 'fewer than half', '1992;22(2):15', 'approximately 30,000', '2003;74(4):309–314', '2015;20(3):311–321', '11.8.6', '07', '48', '174', '16-27', '901', '1959:59(1):1–9', '2007;92(5):821–833', '44-141', '182', '0.4', '2012;83(1):1–6', '2015;132(18', '2011;34(9):1189–1195', 'at least 5,000', '3-100', '1988;108:125–130', '630', '1985,87(2', '521', '2011;20(1):62–66', '2013;178:827–828', '1974;32(1):327–342', '2014;14(3):13–38', '1994;9(5):278–282', 'At least 300', '2009;13:14–17', '10–15', '175', '1776.2', 'Edu-', '3–47', '99–122', '742,800', '2013;13(1):77–85', '5-127', '0.58', '3 to 5', '0.6', '667', 'between 500', "4'8", '30-4', '30 to 40']

DATE ['31 January 2017', '40', 'September 27, 2018', '7-16', '1945–1948', '8-84', 'Prior to 1956', '17 May 1998', 'the past 20 years', '31\nMarch', 'more than 7 days', '15 October 1994', '1964).10', 'under 15 years of age', '1 to 2 nights', 'Between 1955 and 1967', '1980;Art', 'the early\nevening', '2002', '5 decades', '8-44', '8', '


## Explore column groupbys

In [16]:

for file_path, file_path_df in domain_doc_ners_df.groupby('file_path'):
    print(file_path)
    display(file_path_df.sample(4).dropna(axis='columns', how='all').T)
    break

../data/Domain_Knowledge/DoDTR-Data-Dictionary-External.pdf


,4499,5822,19817,4045
file_path,../data/Domain_Knowledge/DoDTR-Data-Dictionary...,../data/Domain_Knowledge/DoDTR-Data-Dictionary...,../data/Domain_Knowledge/DoDTR-Data-Dictionary...,../data/Domain_Knowledge/DoDTR-Data-Dictionary...
nlp_word,\n,at,],210
nlp_tag,_SP,IN,-RRB-,CD
nlp_type,NaN,NaN,NaN,CARDINAL
nlp_pofs,SPACE,ADP,PUNCT,NUM


In [17]:

for nlp_tag, nlp_tag_df in domain_doc_ners_df.groupby('nlp_tag'):
    print(nlp_tag)
    display(nlp_tag_df.sample(4).dropna(axis='columns', how='all').T)
    break

$


,1140912,1223485,1272229,58400
file_path,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Prolonged_Casualty_Ca...
nlp_word,$,$,$,#
nlp_tag,$,$,$,$
nlp_type,MONEY,MONEY,MONEY,NaN
nlp_pofs,SYM,SYM,SYM,SYM


In [22]:

from itertools import combinations

# columns_list = ['bert_entity', 'nlp_tag', 'nlp_type', 'nlp_pofs', 'ent_type']
for groupby_columns in combinations(domain_doc_ners_df.columns, 2):
    groupby_columns = list(groupby_columns)
    df = nu.get_minority_combinations(domain_doc_ners_df, groupby_columns).dropna(axis='columns', how='all')
    if df.shape[0]:
        print(nu.conjunctify_nouns(groupby_columns))
        df.file_path = df.file_path.map(lambda x: str(x).split('/')[-1])
        display(df)

bert_word and bert_entity


,bert_word,bert_entity,bert_score,file_path
630880,May,I-LOC,0.985751,Ethics-ch-02.pdf
1255948,drome,I-MISC,0.857681,Ethics-ch-22.pdf
564170,den,I-ORG,0.938380,Fund ch 6.pdf
590712,MUAL,I-ORG,0.584980,Fund ch 8.pdf


bert_word and bert_score


,bert_word,bert_entity,bert_score,file_path
181027,&,I-ORG,0.901232,Fund ch 15.pdf
497182,Sam,I-LOC,0.993379,Fund ch 39.pdf
65058,Sam,I-LOC,0.993311,SchauerMedicBag.pdf
65107,Sam,I-LOC,0.992895,SchauerMedicBag.pdf


bert_word and file_path


...

,file_path,ent_phrase,ent_type,ent_start,ent_end
998575,Ethics-ch-14.pdf,Nazi,NORP,24497.0,24501.0
233009,Fund ch 2.pdf,US Army Combat Readiness Center,ORG,48399.0,48430.0
252691,Fund ch 21.pdf,US Department of Agriculture,ORG,48394.0,48422.0
158832,Fund ch 12.pdf,Navy,ORG,48390.0,48394.0


ent_type and ent_end


,file_path,ent_phrase,ent_type,ent_start,ent_end
745105,Ethics-ch-05.pdf,European,NORP,24576.0,24584.0
849929,Ethics-ch-09.pdf,Army,ORG,48862.0,48866.0
25009,DoDTR-Data-Dictionary-External.pdf,GCS,ORG,48860.0,48863.0
291517,Fund ch 24.pdf,NASA,ORG,48848.0,48852.0


ent_start and ent_end


,file_path,ent_phrase,ent_type,ent_start,ent_end
472973,Fund ch 36.pdf,Burlington,ORG,63291.0,63301.0
1353592,Ethics-ch-25.pdf,Rwanda,GPE,90370.0,90376.0
805990,Ethics-ch-07.pdf,A11,PERSON,90370.0,90373.0
1022929,Ethics-ch-15.pdf,the Nuremberg Doctors Trial,ORG,90368.0,90395.0
